In [ ]:
import os
import sys
import gmsh
import numpy as np
import import_ipynb
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.constants import epsilon_0
from fem_pre_processing import read_mesh
from fem_processing import gaussian_quadrature, matrices_assembly
from fem_pos_processing import graph_results

# Project 1: Coaxial Problem 

Considere um cabo coaxial com dois dielétricos apresentado na Figura $(1)$.

<figure>
    <img src="pre_processing/pictures/coaxial_problem.png" alt="Fig.1" style="width:20%;" />
    <figcaption>Figure 1: Geometry of Coaxial Problem.</figcaption>
</figure>

O raio do condutor interno é ``a`` e do condutor externo é ``b``. O primeiro dielétrico, $\varepsilon_{1}$, preenche o anel circular entre os raios ``a`` e ``c``, enquanto o segundo dielétrico, $\varepsilon_{2}$, preenche o anel entre os raios ``c`` e ``b``.  

Entre o condutor interno e o externo é aplicada uma diferença de potencial igual a $V$.  

Seja $V = 1$ (potencial zero em ``a`` e um em ``b``), $a = 2 \, mm$, $b = 8 \, mm$, $c = 5 \, mm$, $\varepsilon_{r1} = 2$ e $\varepsilon_{r2} = 4$. Para este problema, além da convergência nas normas $L_2$ e da energia, calcule a convergência para o valor da capacitância do dispositivo.

# `apply_physics()`

In [ ]:
def apply_physics(FINITE_ELEMENT, mesh_data):
    """
    Adiciona uma nova chave 'source' a cada dicionário em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - element_type: Tuple (tipo do elemento, ordem).
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """
    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']

    for key, cell in cell_data.items():
        # Constant material properties
        er = cell['material']['relative_electric_permittivity']
        
        # Adicionar as propriedades dos materiais ao dicionário da célula
        cell['stiffness_term'] = []
        cell['mass_term'] = []
        cell['source'] = []
        
        # Get the global coordinates of the nodes
        ai = [nodes_data[idx]['xg'] for idx in cell['conn']]

        # Get the Gauss points and weights
        gauss_points, _ = gaussian_quadrature.gauss_data(FINITE_ELEMENT)

        # Adicionar a nova chave 'material' ao dicionário da célula
        for xik in gauss_points: 
            xg_e, yg_e = matrices_assembly.isomapping_to_global_coordinates(FINITE_ELEMENT, ai, xik)
            cell['stiffness_term'].append(er * epsilon_0 * np.eye(2))
            cell['mass_term'].append(1)

            # Adicionar a fonte ao dicionário da célula
            cell['source'].append(0)
            
        # Verifica tipo de fronteira absorvente (ABC)
        cell_data[key]['abc'] = {'type': None, 'conn_idx': None}

    return mesh_data

# `coaxial_geometry()`

In [ ]:
def create_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, view_mesh=False, auto_save=True):
    # Define os parâmetros de entrada
    mesh_data = {}
    type, order = FINITE_ELEMENT
    radii = {'a': 2e-3, 'b': 8e-3, 'c': 5e-3}

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("coaxial_cable")
    factory = gmsh.model.occ

    # Criar os círculos com `gmsh.model.occ.addCircle`
    inner_circle = factory.addCircle(0, 0, 0, radii['a'])
    dielectric_circle = factory.addCircle(0, 0, 0, radii['c'])
    outer_circle = factory.addCircle(0, 0, 0, radii['b'])

    # Criar loops das curvas
    inner_loop = factory.addCurveLoop([inner_circle])
    middle_loop = factory.addCurveLoop([dielectric_circle])
    outer_loop = factory.addCurveLoop([outer_circle])

    # Criar superfícies para os dielétricos
    medium_1 = factory.addPlaneSurface([middle_loop, inner_loop])
    medium_2 = factory.addPlaneSurface([outer_loop, middle_loop])

    # Sincronizar geometria
    factory.synchronize()

    # Adicionar grupos físicos para Dim=1
    gmsh.model.addPhysicalGroup(1, [inner_circle], tag=BOUNDARY[0]['tag'], name=BOUNDARY[0]['name'])
    gmsh.model.addPhysicalGroup(1, [outer_circle], tag=BOUNDARY[1]['tag'], name=BOUNDARY[1]['name'])

    # Adicionar grupos físicos para Dim=2
    gmsh.model.addPhysicalGroup(2, [medium_1], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name'])
    gmsh.model.addPhysicalGroup(2, [medium_2], tag=MATERIAL[1]['tag'], name=MATERIAL[1]['name'])

    # Gerar malha 2D
    gmsh.option.setNumber("Mesh.SaveAll", 1)

    # Definir ordem dos elementos
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)
    gmsh.option.setNumber("Mesh.MeshSizeMin", h)
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(order)

    # Visualizar a malha no Gmsh (opcional)
    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        gmsh.write(f"pre_processing/mesh/coaxial_domain_{type}{order}.msh")
        read_mesh.basic_info()

    # Create mesh Structure Data from gmsh
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['edges'] = read_mesh.get_edge_data()
    mesh_data = apply_physics(FINITE_ELEMENT, mesh_data)

    # Finalizar Gmsh
    gmsh.finalize()
    
    return mesh_data

# `set_analytical_solution()`

In [2]:
def set_analytical_solution(x, y):
    # Raio dos condutores e divisão dos dielétricos (em metros)
    a, b, c = 2e-3, 8e-3, 5e-3  
    # Constantes dielétricas
    epsilon_r1, epsilon_r2 = 2, 4  
    # Converter coordenadas cartesianas para polares
    r = np.sqrt(x**2 + y**2)
    
    # Calcular os coeficientes (pré-calculados anteriormente)
    denominator = (epsilon_r1 * np.log(b) - epsilon_r1 * np.log(c) - epsilon_r2 * np.log(a) + epsilon_r2 * np.log(c))
    A1 = epsilon_r2 / denominator
    B1 = -epsilon_r2 * np.log(a) / denominator
    A2 = epsilon_r1 / denominator
    B2 = (-epsilon_r1 * np.log(c) - epsilon_r2 * np.log(a) + epsilon_r2 * np.log(c)) / denominator

    # Verificar a qual região o ponto pertence e calcular o potencial
    # Região 1
    if a <= r <= c:
        u = A1 * np.log(r) + B1
        dV_dr = A1 / r
    # Região 2
    elif c < r <= b:
        u = A2 * np.log(r) + B2
        dV_dr = A2 / r
    else:
        u = 0
        dV_dr = np.array([0.0, 0.0])

    # Gradiente em coordenadas cartesianas
    grad_u = (dV_dr / r) * np.array([x, y])
    
    return u, grad_u

# `capacitance()`

In [ ]:
def capacitance(a, b, c, epsilon_r1, epsilon_r2):
    denominator = (1 / epsilon_r1) * np.log(c / a) + (1 / epsilon_r2) * np.log(b / c)
    return 2 * np.pi * epsilon_0 / denominator

# Exemplo:
V0 = 1.0
a, b, c = 2e-3, 8e-3, 5e-3
epsilon_r1, epsilon_r2 = 2, 4
cap = capacitance(a, b, c, epsilon_r1, epsilon_r2)
print(f"Analytical Capacitance: {cap * 1e12:.2f} pF")
print(f"Total Energy stored in capacitor: {0.5 * cap * V0**2 * 1e12:.2f} pJ")

# `plot_analytical_solution()`

In [ ]:
def plot_analytical_solution(Npts=500):
    # Raio dos condutores e divisão dos dielétricos (em metros)
    a, b, c = 2e-3, 8e-3, 5e-3  
    # Constantes dielétricas
    epsilon_r1, epsilon_r2 = 2, 4  

    # Configurar a grade cartesiana
    x = np.linspace(-b, b, Npts)
    y = np.linspace(-b, b, Npts)
    X, Y = np.meshgrid(x, y)
    R = np.sqrt(X**2 + Y**2)  # Coordenadas polares r
    
    # Calcular os coeficientes
    denominator = (epsilon_r1 * np.log(b) - epsilon_r1 * np.log(c) - epsilon_r2 * np.log(a) + epsilon_r2 * np.log(c))
    A1 = epsilon_r2 / denominator
    B1 = -epsilon_r2 * np.log(a) / denominator
    A2 = epsilon_r1 / denominator
    B2 = (-epsilon_r1 * np.log(c) - epsilon_r2 * np.log(a) + epsilon_r2 * np.log(c)) / denominator

    # Inicializar o potencial com NaN (não definido)
    V = np.full_like(R, np.nan)

    # Calcular o potencial nas regiões do capacitor
    region1 = (R >= a) & (R <= c)
    region2 = (R > c) & (R <= b)
    
    V[region1] = A1 * np.log(R[region1]) + B1
    V[region2] = A2 * np.log(R[region2]) + B2

    # Plotar a distribuição do potencial
    plt.figure(figsize=(8, 8))
    plt.contourf(X, Y, V, levels=100, cmap="viridis")
    plt.colorbar(label="Electric Potential (V)")
    plt.xlabel(r'$x \,(m)$')
    plt.ylabel(r'$y \,(m)$')
    plt.axis("equal")
    plt.tight_layout()

    # Salvar a figura
    filepath = graph_results.get_dir(f"pos_processing/pictures/analytical_solution.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

# `capacitance_convergence()`

In [ ]:
def capacitance_convergence(mesh_sizes, capacitance):
    plt.figure(figsize=(10, 8))
    for element_type, values in capacitance.items():
        type, order = element_type
        plt.semilogx(mesh_sizes, np.array(values)*1E12, marker='o', linestyle='-', label=f'{type} Order {order}')
        
        # Adiciona os valores próximos aos pontos
        for h, value in zip(mesh_sizes, values):
            plt.text(h, value * 1E12 * 1.001, f'{value * 1E12:.4f}', fontsize=8, ha='center')

    plt.xlabel(r'Element size, $h$')
    plt.ylabel(r'Capacitance ($pF$)')
    plt.legend(title="Element Types")
    plt.tight_layout()

    # Salvando o arquivo no formato SVG
    filepath = graph_results.get_dir(f"pos_processing/pictures/convergence_capacitance.svg")
    print(f"Arquivo salvo em: {filepath}")
    plt.savefig(filepath, format="svg")
    plt.close()

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.